***

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys, os
sys.path.append(r'C:\dev\code\gmt-amp\python-analytics-platform')
sys.path.append(r'C:\dev\code\gmt-amp\amp-models')

In [3]:
from gmt.dna.datagateway.data_adapter_factory import DataAdapterFactory

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from datetime import datetime, date
import pandas as pd

# SQL

In [6]:
sql_query_many_curves = f"""
SELECT d.[CurveName]
	,datefromparts(year(min(d.PointDate)), month(min(d.PointDate)), 1) AS PointDate
	,AVG(d.[PointValue]) AS PointValue
FROM [arc_ts_data].[PriceCurveData] d
WHERE d.EffectiveDate = '2018-01-02'
	AND [CurveName] IN (
		'G_H_NBP.USD'
		,'G_H_Zeebrugge-Hub.Pnc'
		,'LNG_JKM_M.USD'
		,'G_H_TRS.EUR'
		,'LNG_DES_ME_W_INDIA_M.USD'
		,'LNG_H_DES_Med_MID_M.USD'
		,'LNG_H_DES_JKM_MID_M.USD'
		,'LNG_H_DES_NWE_MID_M.USD'
		,'G_H_TTF.EUR'
		,'G_H_CEGH.EUR'
		,'G_H_AOC.EUR'
		,'G_H_PEG-Nord.EUR'
		,'G_H_PSV.EUR'
		,'G_NYM_HH_M.USD'
		,'LNG_H_DES_Lat_Am_MID_M.USD'
		,'FX_curve_cross_EUR.USD'
		,'FX_curve_input_USD.GBP'
		,'FREIGHT_H_LNG_M_RolledOver.USD'
		)
	AND d.PointDate >= '2018-02-01'
	AND d.PointDate < '2021-02-01'
GROUP BY d.CurveId
	,d.CurveName
	,d.EffectiveDate
	,12 * year(d.PointDate) + month(d.PointDate)
ORDER BY d.CurveID
"""

In [7]:
sql_query_one_curve = f"""
SELECT d.[CurveName]
	,datefromparts(year(min(d.PointDate)), month(min(d.PointDate)), 1) AS PointDate
	,AVG(d.[PointValue]) AS PointValue
FROM [arc_ts_data].[PriceCurveData] d
WHERE d.EffectiveDate = '2018-01-02'
	AND [CurveName] IN (
		'G_H_NBP.USD'
		,'G_H_Zeebrugge-Hub.Pnc'
		,'LNG_JKM_M.USD'
		,'G_H_TRS.EUR'
		,'LNG_DES_ME_W_INDIA_M.USD'
		,'LNG_H_DES_Med_MID_M.USD'
		,'LNG_H_DES_JKM_MID_M.USD'
		,'LNG_H_DES_NWE_MID_M.USD'
		,'G_H_TTF.EUR'
		,'G_H_CEGH.EUR'
		,'G_H_AOC.EUR'
		,'G_H_PEG-Nord.EUR'
		,'G_H_PSV.EUR'
		,'G_NYM_HH_M.USD'
		,'LNG_H_DES_Lat_Am_MID_M.USD'
		,'FX_curve_cross_EUR.USD'
		,'FX_curve_input_USD.GBP'
		,'FREIGHT_H_LNG_M_RolledOver.USD'
		)
	AND d.PointDate >= '2018-02-01'
	AND d.PointDate < '2021-02-01'
GROUP BY d.CurveId
	,d.CurveName
	,d.EffectiveDate
	,12 * year(d.PointDate) + month(d.PointDate)
ORDER BY d.CurveID
"""

In [8]:
import sqlalchemy as sa
def query_arc_alchemy(sql):    
    _url ='mssql+pyodbc:///?odbc_connect=DATABASE%3DARCTIMESERIES%3BDRIVER%3D%7BSQL+Server+Native+Client+11.0%7D%3BSERVER%3DARCSQL%3BTrace%3D0%3BTrusted_Connection%3DYes'
    engine =sa.create_engine(_url, echo=False, convert_unicode=True)
    with engine.connect() as connection:
        result =connection.execute(sql)
        data =result.fetchall()
        column_names =result.keys()
        dataframe = pd.DataFrame.from_records(data, columns=column_names)
        return dataframe

## Testing one curve:

In [9]:
%timeit _ = query_arc_alchemy(sql_query_one_curve)

The slowest run took 4.01 times longer than the fastest. This could mean that an intermediate result is being cached.
245 ms ± 115 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Testing many curves:

In [10]:
%timeit _ = query_arc_alchemy(sql_query_many_curves)

190 ms ± 35.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## HTTP

In [11]:
t = date(2018, 1, 2);
from_date = date(2018,2,1)
to_date = date(2021,2,1)

In [12]:
arc_curves = ['G_H_NBP.USD',
 'G_H_Zeebrugge-Hub.Pnc',
 'LNG_JKM_M.USD',
 'G_H_TRS.EUR',
 'LNG_DES_ME_W_INDIA_M.USD',
 'LNG_H_DES_Med_MID_M.USD',
 'LNG_H_DES_JKM_MID_M.USD',
 'LNG_H_DES_NWE_MID_M.USD',
 'G_H_TTF.EUR',
 'G_H_CEGH.EUR',
 'G_H_AOC.EUR',
 'G_H_PEG-Nord.EUR',
 'G_H_PSV.EUR',
 'G_NYM_HH_M.USD',
 'LNG_H_DES_Lat_Am_MID_M.USD',
 'FX_curve_cross_EUR.USD',
 'FX_curve_input_USD.GBP',
 'FREIGHT_H_LNG_M_RolledOver.USD']

In [13]:
def http_query_arc(curves, t, from_date, to_date):
    data_adaptor = DataAdapterFactory.create("arc", env='prod')
    for curve in curves:
        _ = data_adaptor.get_forward_curve(curve, t)

## Testing one curve

In [14]:
%timeit http_query_arc(arc_curves[:1], t, from_date, to_date)

297 ms ± 91.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Testing many curves

In [15]:
%timeit http_query_arc(arc_curves, t, from_date, to_date)

4.5 s ± 510 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
